In [2]:
from bonsai.data_loaders import load_data
from bonsai.net_vert import Net
from bonsai.trainers import *
from bonsai.helpers import *

%load_ext autoreload
%autoreload 2

In [108]:
options = [0,0,12,120]
goal = .5
print(sum(options)*goal)
weights = [(sum(options)-x)/sum(options) for x in options]
weights

66.0


[1.0, 1.0, 0.9090909090909091, 0.09090909090909091]

In [126]:
n=5
sums=[]
for i in range(n):
    used,chosen = 0,[]
    for option in options:
        if np.random.rand()<goal:
            chosen.append(option)
            used+=option
    #print(chosen)
    sums.append(used)
np.mean(sums),np.std(sums)

(66.046884, 60.29554578814709)

In [3]:
gpu_space = 8.2
batch_size = 256*4
hypers = {
    'dataset':'CIFAR10',
    'classes':10,
    'scale':2,
    'batch_size':batch_size,
    'multiplier':1024//batch_size,
    'num_patterns':1,
    'patterns':[['r','r','ra']],
    'nodes':4,
    'half':False,
    'lr_schedule':
        {'lr_min': 0,
         'lr_max': 1e-2,
         't_0': 600,
         't_mult': 1},
    'epochs':600,
    'drop_prob':.25,
}
schedule = {'cycle_len':4,
            'transition_after':2,
            'n_cycles':4}
data, data_shape = load_data(hypers['batch_size'], hypers['dataset'])

In [20]:
model = Net(dim=data_shape, 
            classes=hypers['classes'], 
            scale=hypers['scale'],
            patterns=hypers['patterns'], 
            num_patterns=1,
            nodes=hypers['nodes'],
            auxiliary=True,
            random_ops={'e_c':.5,'i_c':1,'worst_case':True},
            half=hypers['half'])
model.genotype_compression()

{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0
{'Identity': 0, 'Max_Pool_3x3': 0, 'Conv_1x1': 12, 'Sep_Conv_3x3': 120}
{} 0

(nan, 1.0)

In [5]:
full_train(
    model,
    data,
    resume=False,
    epochs=4,
    drop_prob=hypers['drop_prob'],
    comp_lambdas=TransitionDict({0: {'edge': 1, 'input': 1}}),
    comp_ratio=.5,/
    prune_interval=schedule['cycle_len'],
    lr_schedule=schedule_generator(hypers['lr_schedule'])(32),
    half=hypers['half']
);
model.add_pattern()

=== Training Doge Finnish Marie ===
Starting at 2019-10-25 14:01:14.368596
10/25/2019 02:01 PM
Init: 46.00MiB
0: 3.33GiB
1: 5.31GiB
2: 5.96GiB
GP: 5.96GiB
Classifier: 5.96GiB
Train Epoch: 0   [41552 /50000  (100%)]	Comp Ratio: [E: 0.361, I: 1.333], Loss Comp: [C: 2.482, E: 0.298, I: 0.33], Losses [0: 1.85]Per Epoch: 31.68s , Alloc: 5.99GiB  
Train Corrects: Top-1: 29.14%, Comp: 0.36, 1.33 44.04s
Train Loss Components: C: 2.482, E: 0.298, I: 0.33
MGC: tensor(1., device='cuda:0')
All Towers Test  Corrects: Top-1: 39.00%, 1.96s
Last Tower Test  Corrects: Top-1: 39.00%, 1.96s

Adjusting lr to 0.01
10/25/2019 02:02 PM
Train Epoch: 1   [41552 /50000  (100%)]	Comp Ratio: [E: 0.469, I: 1.333], Loss Comp: [C: 2.163, E: 0.114, I: 0.33], Losses [0: 1.72]Per Epoch: 32.69s , Alloc: 5.99GiB  
Train Corrects: Top-1: 37.01%, Comp: 0.47, 1.33 44.82s
Train Loss Components: C: 2.163, E: 0.114, I: 0.33
MGC: tensor(1., device='cuda:0')
All Towers Test  Corrects: Top-1: 42.69%, 1.99s
Last Tower Test  Correc

In [40]:
print([sum([general_num_params(op) for k,edge in cell.edges.items() for op in edge.ops]) for cell in model.cells])

[1904, 4648, 12824]
